In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from openpyxl import load_workbook
import numpy as np
#가게 이름과 해당 가게의 메뉴 리스트가 담긴 엑셀
tempEx=load_workbook(filename='/content/gdrive/My Drive/Colab Notebooks/voice_order/data/store_menu.xlsx')
sheet1=tempEx['Sheet1']
menu=[]
tp_menu=[]
op_menu=[]
#가게 이름
store_name='미식반점'
for i in sheet1.rows:
        k=i[0].value
        if k in store_name:
          menu.append(i[1].value)
          tp_menu.append(i[2].value)
          op_menu.append(i[3].value)
          
for i in range(len(tp_menu)):
  if(tp_menu[i]==None):
    continue
  if(tp_menu[i].find(',')!=-1):
    tp_menu[i]=tp_menu[i].split(',')
for i in range(len(op_menu)):
  if(op_menu[i]==None):
    continue
  op_menu[i]=str(op_menu[i])  
  if(str(op_menu[i]).find(',')!=-1):
    print(op_menu[i])
    op_menu[i]=(op_menu[i]).split(',')
print(op_menu)

소,중
소,중
소,중
땅콩소스,매운소스
125,250
250,500
280,560
[['소', '중'], ['소', '중'], ['소', '중'], None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, ['땅콩소스', '매운소스'], None, ['125', '250'], ['250', '500'], ['280', '560'], None, None, None, None, None, None, None, None, None, None, None, None]


# 네이버 clova speech recognition

In [ ]:
import sys
import requests

!pip install pymysql
import pymysql
import os
import sys
sys.path.insert(0, '/content/gdrive/My Drive/Colab Notebooks/voice_order/data/')
import config
client_id=config.DATABASE_CONFIG['client_id']
client_secret=config.DATABASE_CONFIG['client_secret']

lang = "Kor" # 언어 코드 ( Kor, Jpn, Eng, Chn )
url = "https://naveropenapi.apigw.ntruss.com/recog/v1/stt?lang=" + lang
data = open('/content/gdrive/My Drive/Colab Notebooks/voice_order/data/e6.wav', 'rb')

headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret,
    "Content-Type": "application/octet-stream"
}
response = requests.post(url,  data=data, headers=headers)
rescode = response.status_code
if(rescode == 200):
    print (response.text)
else:
    print("Error : " + response.text)
txt=response.text
txt1=txt[9:-2]
print(txt1)

     |████████████████████████████████| 51kB 997kB/s 
{"text":"고추잡채 안하고 영상 피안하고 잡채밥 하나 중식초개 냉면 두개 주세요 중식초계냉면 하나는 땅콩소스로 하나는 매운소스로 주세요"}
고추잡채 안하고 영상 피안하고 잡채밥 하나 중식초개 냉면 두개 주세요 중식초계냉면 하나는 땅콩소스로 하나는 매운소스로 주세요


# ETRI 음성인식

In [ ]:
#-*- coding:utf-8 -*-
import urllib3
import json
import base64
openApiURL="http://aiopen.etri.re.kr:8000/WiseASR/Recognition"
accessKey="key입력"
audioFilePath='/content/gdrive/My Drive/Colab Notebooks/voice_order/data/e6.wav'
languageCode='korean'



file = open(audioFilePath, "rb")
audioContents = base64.b64encode(file.read()).decode("utf8")
file.close()

requestJson={
    'access_key':accessKey,
    'argument':{
        'language_code': languageCode,
        'audio': audioContents
    }
}


http=urllib3.PoolManager()
response=http.request(
    'POST',
    openApiURL,
    headers={"Content-Type":"application/json; charset=UTF-8"},
    body=json.dumps(requestJson)
)
print('[responseCode]'+str(response.status))
print('[responBody]')
print((response.data).decode('utf8'))
txt=(response.data).decode('utf8')
txt2=txt[43:-3]
print(txt2)

[responseCode]200
[responBody]
{"result":0,"return_object":{"recognized":"고추 잡채 하나 꼬 양장피 하나 꼬 잡채밥 하나 꼬 중식 초계 냉면 2개 주세요 중식 초계 냉면 하나는 땅콩 소스로 하나는 매운 소스로 주세요 "}}
고추 잡채 하나 꼬 양장피 하나 꼬 잡채밥 하나 꼬 중식 초계 냉면 2개 주세요 중식 초계 냉면 하나는 땅콩 소스로 하나는 매운 소스로 주세요 


In [ ]:
!pip install hgtk
import hgtk
import difflib
num_list=[['1','한','하나','원'],['2','두','둘','투'],[' 세','셋','쓰리'],[' 네','넷','포'],['다섯','파이브'],['여섯','식스']
          ,['일곱','세븐'],['여덟','에잇'],['아홉','나인'],['열','텐 '],['열하나','열한'],['열둘','열두']
          ,['열셋','열세'],['열넷','열네'],['열다섯'],['열여섯'],['열일곱'],['열여덟'],['열아홉'],['스물','스무']]




rate=-999
f_rate=-999
r_txt1=hgtk.text.decompose(txt1) #네이버 음성인식 결과의 초중성을 분리하여 저장
r_txt2=hgtk.text.decompose(txt2) #ETRI 음성인식 결과의 초중성을 분리하여 저장
ttt=''
max1=0
all_rate1=0
f_st=0
cnt1=0
cnt2=0
t_m=[]
for i in range(len(menu)):
  r_menu=hgtk.text.decompose(menu[i]) #r_menu->메뉴의 초중성을 분리한 결과
  st=0
  ed=len(r_menu)
  while(1):
    if(ed>len(r_txt1)):
      break
    ttt=r_txt1[st:ed] #본 문장에서 r_menu와 같은 길이만큼 뗌
    rate=difflib.SequenceMatcher(None,r_menu,ttt).ratio() #r_menu와 얼마나 비슷한지 비율로 저장
    if(rate>0.8): #0.8 이상이면 비슷하다고 판단
      all_rate1=all_rate1+rate 
    st=st+1
    ed=ed+1
all_rate2=0

print(all_rate1)
for i in range(len(menu)):
  r_menu=hgtk.text.decompose(menu[i])
  st=0
  ed=len(r_menu)
  while(1):
    if(ed>len(r_txt2)):
      break
    ttt=r_txt2[st:ed]
    f_rate=rate
    rate=difflib.SequenceMatcher(None,r_menu,ttt).ratio()
    if(rate>0.8):
      all_rate2=all_rate2+rate
    
    st=st+1
    ed=ed+1
print(all_rate1)
print(all_rate2)

#네이버와 ETRI의 결과 중 메뉴와 비슷한 글자가 더 많은 결과를 사용 
if(all_rate1>all_rate2):
  r_txt=r_txt1
else:
  r_txt=r_txt2
  
print('1:',hgtk.text.compose(r_txt1),'2:',hgtk.text.compose(r_txt2))  
print('r_txt:',hgtk.text.compose(r_txt))  
  
for i in range(len(menu)):
  r_menu=hgtk.text.decompose(menu[i]) 
  st=0 #탐색을 시작할 부분
  ed=len(r_menu) #탐색을 끝낼 부분
  while(1):
    if(ed>len(r_txt)):
      break
    ttt=r_txt[st:ed]
    
    rate=difflib.SequenceMatcher(None,r_menu,ttt).ratio()
    
    if(rate>0.88):
      r_txt=r_txt.replace(r_txt[st:ed],(r_txt[st:ed].replace(' ',''))) #메뉴이름 사이의 공백 삭제
      r_txt=r_txt.replace(r_txt[st:ed],r_menu) #메뉴 철자 수정
      t_m.append(i) #메뉴 인덱스를 t_m(tmp_menu) 배열에 저장
      break
    
    st=st+1
    ed=ed+1

for ij in range(len(t_m)):
  if(op_menu[t_m[ij]]==None):
    continue
  for i in range(len(op_menu[t_m[ij]])):
    r_menu=hgtk.text.decompose(op_menu[t_m[ij]][i])
    st=0
    sum_cnt=0
    ed=len(r_menu)
    while(1):
      if(ed>len(r_txt)):
        break
      ttt=r_txt[st:ed]
      rate=difflib.SequenceMatcher(None,r_menu,ttt).ratio()
      if(rate>0.80):
        r_txt=r_txt.replace(r_txt[st:ed],(r_txt[st:ed].replace(' ','')))
        print(hgtk.text.compose(r_menu),hgtk.text.compose(r_txt))
        r_txt=r_txt.replace(r_txt[st:ed],r_menu)
        break
      st=st+1
      ed=ed+1    

r_txt=hgtk.text.compose(r_txt)
print(r_txt)

    
    
st_p=0
#메뉴를 기준으로 띄어쓰기를 삽입
for i in range(len(menu)):
  u=r_txt.find(menu[i],st_p)
  if(u!=-1):
    final_u=u+len(menu[i])+1
    if(u!=0):
      start_u=u-1
      if(r_txt[start_u]!=' '):
        r_txt=r_txt[:start_u+1]+' '+r_txt[start_u+1:]
    else:
      start_u=u
      
#토핑메뉴를 기준으로 띄어쓰기를 삽입
for ij in range(len(t_m)):
  if(op_menu[t_m[ij]]==None):
    continue
  for i in range(len(op_menu[t_m[ij]])):
    if(len(op_menu[t_m[ij]][i])==1):
      u=r_txt.find(' '+op_menu[t_m[ij]][i],st_p)
    else:
      u=r_txt.find(op_menu[t_m[ij]][i],st_p)
    if(u!=-1):
      final_u=u+len(op_menu[t_m[ij]][i])+1
      if(u!=0):
        start_u=u
        if(r_txt[start_u-1]!=' '):
          r_txt=r_txt[:start_u]+' '+r_txt[start_u:]
      else:
        start_u=u

#수량을 기준으로 띄어쓰기 삽입        
for i in range(len(num_list)):
  for j in range(len(num_list[i])):
    u=r_txt.find(' '+num_list[i][j])
    if(u!=-1):
      if((u+1<len(r_txt) and r_txt[u+1]>='1' and r_txt[u+1]<='9') or (u-1>=0 and r_txt[u-1]>='1' and r_txt[u-1]<='9')):
        continue
      final_u=u+len(num_list[i][j])+1
      if(u!=0):
        start_u=u+1
        print(start_u,final_u)
        if(r_txt[start_u-1]!=' '):
          r_txt=r_txt[:start_u-1]+' '+r_txt[start_u-1:]
        if(r_txt[final_u]!=' '):
          r_txt=r_txt[:final_u]+' '+r_txt[final_u:]  
          
      else:
        start_u=u
      
print(r_txt) 


  Created wheel for hgtk: filename=hgtk-0.1.3-py2.py3-none-any.whl size=6689 sha256=9b5b8ec144cd18124adebf3f2632795791677fba2051e84e687dc8f0e9cdfb1d
  Stored in directory: /root/.cache/pip/wheels/73/72/06/6065a57fe68264f35d7e52e37f56831eb3e9ec75656880de20
Successfully built hgtk
20.49650349650349
20.49650349650349
24.601398601398596
1: 고추잡채 안하고 영상 피안하고 잡채밥 하나 중식초개 냉면 두개 주세요 중식초계냉면 하나는 땅콩소스로 하나는 매운소스로 주세요 2: 고추 잡채 하나 꼬 양장피 하나 꼬 잡채밥 하나 꼬 중식 초계 냉면 2개 주세요 중식 초계 냉면 하나는 땅콩 소스로 하나는 매운 소스로 주세요 
r_txt: 고추 잡채 하나 꼬 양장피 하나 꼬 잡채밥 하나 꼬 중식 초계 냉면 2개 주세요 중식 초계 냉면 하나는 땅콩 소스로 하나는 매운 소스로 주세요 
땅콩소스 고추잡채 하나 꼬양장피 하나 꼬잡채밥 하나 꼬 중식초계냉면 2개 주세요 중식초계냉면 하나는땅콩소스로 하나는 매운 소스로 주세요 
매운소스 고추잡채 하나 꼬양장피 하나 꼬잡채밥 하나 꼬 중식초계냉면 2개 주세요 중식초계냉면 하나는땅콩소스로 하나는매운소스로 주세요 
고추잡채 하나 꼬양장피 하나 꼬잡채밥 하나 꼬 중식초계냉면 2개 주세요 중식초계냉면 하나는땅콩소스로 하나는매운소스로 주세요 
5 7
고추잡채 하나 꼬 양장피 하나 꼬 잡채밥 하나 꼬 중식초계냉면 2개 주세요 중식초계냉면 하나는 땅콩소스로 하나는 매운소스로 주세요 


# 메뉴 전체적으로 몇 개인가 카운트

In [ ]:
from openpyxl import load_workbook
import numpy as np
import operator
#가게 이름과 해당 가게의 메뉴 리스트가 담긴 엑셀
tempEx=load_workbook(filename='/content/gdrive/My Drive/Colab Notebooks/voice_order/data/store_menu.xlsx')
sheet1=tempEx['Sheet1']
menu=[]
tp_menu=[]
op_menu=[]
#가게 이름
store_name='미식반점'

num_list=[['한 ','하나','원'],[' 2','두 ','둘','투'],['세 ','셋','쓰리'],['네 ','넷','포'],['다섯 ','다섯','파이브'],['여섯 ','여섯','식스']
          ,['일곱 ','일곱','세븐'],['여덟 ','여덟','에잇'],['아홉 ','아홉','나인'],['열 ','텐 '],['열하나','열한 '],['열둘','열두 ']
          ,['열셋','열세 '],['열넷','열네 '],['열다섯'],['열여섯'],['열일곱'],['열여덟'],['열아홉'],['스물','스무 ']]

all_menu_dict={}

#수량을 묶어 세는 경우를 확인하기 위한 부사 list
all_list=['전부','모두','씩']

#문장이 끊기는 단위를 확인하기 위한 접속사 list
and_list=['고 ','요 ']

#단어를 엮는 조사
pp_list=['랑 ']

#해당 음식점의 메뉴를 불러와 전체 메뉴 리스트를 생성
for i in sheet1.rows:
        k=i[0].value
        if k in store_name:
          menu.append(i[1].value)
          tp_menu.append(i[2].value)
          op_menu.append(i[3].value)
for i in range(len(menu)):
  all_menu_dict[menu[i]]=i

#주문을 mylist에 받아옴          
#with open('/content/gdrive/My Drive/Colab Notebooks/stt/e_c_6_2.txt','rb') as f:
    #mylist=[f.read().decode('euc-kr')]
#print(mylist)

mylist=[r_txt]
print(mylist)
#menu_list->주문이 들어갈 메뉴들 집합
menu_list={}

#num_index->수량 확인을 위해 메뉴가 말해진 구간을 확정지음
num_index={}

#첫 단어가 음식 이름일 경우
for i in range(0,np.shape(menu)[0]):
  t=menu[i]
  u=mylist[0].find(t)
  
  if(u==0):
    num_index[t]=[u,u+len(t)]
    menu_list[t]=0
    
#그 이후부터    
for i in range(0,np.shape(menu)[0]):
  t=menu[i]
  u=mylist[0].find(' '+t)
  if(u!=-1):
    if(menu_list.get(t)!=None):
      continue
    menu_list[t]=0
    num_index[t]=[u+1,u+len(t)+1]

l_menu_list=list(menu_list)

sortedarr=sorted(num_index.items(),key=operator.itemgetter(1))

num_index={}
menu_list={}
for i in range(len(sortedarr)):
  num_index[sortedarr[i][0]]=sortedarr[i][1]
  menu_list[sortedarr[i][0]]=0
l_menu_list=list(menu_list)

for i in range(len(num_index)-1):
  for j in range(i+1,len(num_index)):
    if(num_index[l_menu_list[i]][0]==num_index[l_menu_list[j]][0] or num_index[l_menu_list[i]][1]==num_index[l_menu_list[j]][0]):
      if(num_index[l_menu_list[i]][1]-num_index[l_menu_list[i]][0]>num_index[l_menu_list[j]][1]-num_index[l_menu_list[j]][0]):
        del(menu_list[l_menu_list[j]])
        del(num_index[l_menu_list[j]])
      else:
        del(menu_list[l_menu_list[i]])
        del(num_index[l_menu_list[i]])

l_menu_list=list(menu_list)

#메뉴와 메뉴 사이의 구간에서 수량 표현을 찾아 menu_list에 append함
for i in range(len(num_index)):
  flag=0
  Max=0
  idx=-1
  for j in range(len(num_list)):
    for w in range(len(num_list[j])):
      if(i+1<len(num_index)):
        result=mylist[0][num_index[l_menu_list[i]][1]:num_index[l_menu_list[i+1]][0]].find(num_list[j][w])
      else:
        result=mylist[0][num_index[l_menu_list[i]][1]:].find(num_list[j][w])
      if(result!=-1):
        if(j+1>Max):
          Max=j+1
  menu_list[l_menu_list[i]]=Max

#수량이 0인 메뉴들을 zero_menu에 append
zero_menu=[]
for i in range(len(menu_list)):
  if(menu_list[l_menu_list[i]]==0):
    zero_menu.append(i)
    
s_index=[]
e_index=[]
s_index.append(0)
i_flag=0
#부사 표현이 들어간 리스트의 인덱스를 저장
for i in range(len(menu_list)):
  for j in range(len(all_list)):
    a=all_list[j]  
    if(mylist[0].find(a)!=-1):
      if(len(e_index)!=0 and mylist[0].find(a) in e_index):
        continue
      e_index.append(mylist[0].find(a))

#접속사 표현이 들어간 리스트의 인덱스를 저장      
for i in range(len(menu_list)):
  for j in range(len(and_list)):
    a=and_list[j]
    if(mylist[0].find(a)!=-1):
      if(len(s_index)!=0 and mylist[0].find(a) in s_index):
        continue
      s_index.append(mylist[0].find(a))


#부사 표현이 들어간 인덱스와 접속사 표현이 들어간 인덱스들 사이에 있는 메뉴를 revise_list에 저장      
revise_list=[]
for i in range(len(l_menu_list)):
  t=l_menu_list[i]
  for j in range(len(e_index)):
    print(mylist[0][s_index[j]:e_index[j]])
    u=mylist[0][s_index[j]:e_index[j]].find(t)
    if(u!=-1):
      revise_list.append([t])

#revise_list 다음에 나오는 메뉴의 수량을 revise_list의 메뉴들에 적용
for i in range(len(menu_list)):
  if(len(revise_list)==0):
    break
  if(revise_list[len(revise_list)-1][0]==l_menu_list[i]):
    new_num=menu_list[l_menu_list[i]]
for i in range(len(revise_list)):
  for j in range(len(menu_list)):
    if(revise_list[i][0]==l_menu_list[j]):
      menu_list[l_menu_list[j]]=new_num

#수량표현이 없는 메뉴들의 수량을 0에서 1로 변경      
for i in range(len(menu_list)):
  if(menu_list[l_menu_list[i]]==0):
    menu_list[l_menu_list[i]]=1
  
print(menu_list)

#토핑 메뉴 저장
for i in range(len(tp_menu)):
  if(tp_menu[i]==None):
    continue
  if(tp_menu[i].find(',')!=-1):
    tp_menu[i]=tp_menu[i].split(',')
for i in range(len(op_menu)):
  if(op_menu[i]==None):
    continue
  op_menu[i]=str(op_menu[i])  
  if(str(op_menu[i]).find(',')!=-1):
    
    op_menu[i]=(op_menu[i]).split(',')


['고추잡채 하나 꼬 양장피 하나 꼬 잡채밥 하나 꼬 중식초계냉면 2개 주세요 중식초계냉면 하나는 땅콩소스로 하나는 매운소스로 주세요 ']
{'고추잡채': 1, '양장피': 1, '잡채밥': 1, '중식초계냉면': 2}


# 옵션 메뉴 추가


In [ ]:

word_tmp=[] #[[메뉴이름],[텍스트에서의 위치]]
cnt=0
for i in range(len(l_menu_list)):
  for j in range(mylist[0].count(l_menu_list[i])):
    if(j==0):
      word_tmp.append([l_menu_list[i],[mylist[0].find(l_menu_list[i]),mylist[0].find(l_menu_list[i])+len(l_menu_list[i])]])
      
    else:
      word_tmp.append([l_menu_list[i],[word_tmp[cnt-1][1][1]+mylist[0][word_tmp[cnt-1][1][1]:].find(l_menu_list[i]),word_tmp[cnt-1][1][1]+mylist[0][word_tmp[cnt-1][1][1]:].find(l_menu_list[i])+len(l_menu_list[i])]])
      
    cnt=cnt+1

print(word_tmp)

n_word_tmp=[] #위치순으로 정렬
for i in range(len(word_tmp)):
  r_min=9999
  r_idx=0
  for j in range(len(word_tmp)):
    if(r_min>word_tmp[j][1][0]):
      r_min=word_tmp[j][1][0]
      r_idx=j
  n_word_tmp.append(word_tmp[r_idx])
  del(word_tmp[r_idx])

print('n_word_tmp: ',n_word_tmp)
temp_section=[]
real_list=[]
temp_list=[]
l_flag=0
tmp_n_list=[]
real_tmp_n_list=[]

#단어를 엮는 조사를 기준으로 메뉴를 합쳐 텍스트 내에서 구간을 확정지음
for i in range(len(n_word_tmp)):
  if(i+1<len(n_word_tmp)):
    l_flag=0
    temp_section=mylist[0][n_word_tmp[i][1][0]:n_word_tmp[i+1][1][1]]
  else:
    temp_section=mylist[0][n_word_tmp[i][1][0]:]
  print('t:',temp_section)
  for j in range(len(pp_list)):
      
    if(temp_section.find(pp_list[j])!=-1):
      if(op_menu[all_menu_dict[n_word_tmp[i][0]]]==None):
        continue
      for k in range(len(op_menu[all_menu_dict[n_word_tmp[i][0]]])):
          
        if(temp_section[0:temp_section.find(pp_list[j])].find(op_menu[all_menu_dict[n_word_tmp[i][0]]][k])!=-1):
          l_flag=1
          temp_list.append(n_word_tmp[i][0])
          tmp_n_list.append(n_word_tmp[i][1][0])
          tmp_n_list.append(n_word_tmp[i][1][1])
          real_list.append(temp_list)
          real_tmp_n_list.append(tmp_n_list)
          break
      if(tp_menu[all_menu_dict[n_word_tmp[i][0]]]==None):
        continue
      for k in range(len(tp_menu[all_menu_dict[n_word_tmp[i][0]]])):
        if(temp_section[0:temp_section.find(pp_list[j])].find(tp_menu[all_menu_dict[n_word_tmp[i][0]]][k])!=-1):
          l_flag=1
          temp_list.append(n_word_tmp[i][0])
          tmp_n_list.append(n_word_tmp[i][1][0])
          tmp_n_list.append(n_word_tmp[i][1][1])
          real_list.append(temp_list)
          real_tmp_n_list.append(tmp_n_list)
          break
          
            
      if(l_flag==1):
        break
      if(i+1<len(n_word_tmp)):
        temp_list.append(mylist[0][n_word_tmp[i][1][0]:n_word_tmp[i+1][1][1]-len(n_word_tmp[i+1][0])])
        tmp_n_list.append(n_word_tmp[i][1][0])
      
    else:
      
      temp_list.append(n_word_tmp[i][0])
      tmp_n_list.append(n_word_tmp[i][1][0])
      tmp_n_list.append(n_word_tmp[i][1][1])
      real_list.append(temp_list)
      real_tmp_n_list.append(tmp_n_list)
      temp_list=[]  
      tmp_n_list=[]
      
      
r_real_list=[]   
real_sentence=''
for i in range(len(real_list)):
  for j in range(len(real_list[i])):
    real_sentence+=real_list[i][j]
  r_real_list.append(real_sentence)
  real_sentence=''
      
print('r_real_list:',r_real_list)
r_real_list_idx=[]
temp_list=[]
for i in range(len(real_tmp_n_list)):
  a=real_tmp_n_list[i][0]
  b=real_tmp_n_list[i][len(real_tmp_n_list[i])-1]
  real_tmp_n_list[i]=[a,b]
print('real_tmp_n_list:',real_tmp_n_list)
whole_dict={}
whole_tmp=[]
final_op_dict={}

word_min=9999
final=[]
l_n_word_tmp=list(n_word_tmp)
for i in range(len(real_tmp_n_list)):
  
  if(i+1<len(real_tmp_n_list)):
    section=mylist[0][real_tmp_n_list[i][0]:real_tmp_n_list[i+1][0]]
  else:
    section=mylist[0][real_tmp_n_list[i][0]:]
  print('sec:',section)
  if(r_real_list[i] not in all_menu_dict):
    for h in range(len(n_word_tmp)):
      if(section.find(n_word_tmp[h][0])!=-1): #한 구간 사이에 옵션 추가가 있나를 확인함
        t_index=all_menu_dict[n_word_tmp[h][0]]
        if(op_menu[t_index]==None): #옵션 메뉴가 없으면 넘어감
          continue
        for h1 in range(len(op_menu[t_index])):
          if(section.find(op_menu[t_index][h1])!=-1):
            for k in range(len(num_list)):
              for w in range(len(num_list[k])):
                if(section.find(num_list[k][w])!=-1):
                  tt=(u)-section.find(num_list[k][w])

                  if(word_min>abs(tt)):
                    word_min=abs(tt)
                    nn=k+1
                    n_flag=1
            final.append([n_word_tmp[h][0],op_menu[t_index][h1],[nn]]) #final에 메뉴 이름, 옵션, 수량을 추가함
    continue          
  ij=all_menu_dict[r_real_list[i]]
  if(op_menu[ij]==None):
    continue
  for j in range(len(op_menu[ij])):
    n_flag=0
    n_min=9999
    nn=0
    word_min=9999
    u=section.find(' '+op_menu[ij][j])
    if(u!=-1):
      for k in range(len(num_list)):
        for w in range(len(num_list[k])):
          if(section.find(num_list[k][w])!=-1):
            tt=(u)-section.find(num_list[k][w])
            #메뉴 이름과 가장 가까운 수량을 사용
            if(word_min>abs(tt)):
              word_min=abs(tt)
              nn=k+1
              n_flag=1
      final.append([r_real_list[i],op_menu[ij][j],[nn]])
        

final_count=0
nn=0
for i in range(len(final)):
  final_count=0
  nn=0
  t1=0
  for j in range(len(l_menu_list)):
    final_count+=final[i].count(l_menu_list[j])
  if(final_count!=1):
    for k in range(len(num_list)):
      for w in range(len(num_list[k])):
        u1=final[i][0].find(num_list[k][w])
        if(u1!=-1):
          t1=final[i][2][0]
          nn=k+1
    final[i][2]=[nn,t1]

for i in range(len(final)):
  t=[]
  for j in range(len(menu_list)):
    if(final[i][0].find(l_menu_list[j])!=-1):
      t.append(l_menu_list[j])
  final[i][0]=t
    

final_list=[]
#json 파일을 만들기 위하여 리스트 수정
for i in range(len(final)):
  n_min=9999
  nn=0
  for j in range(len(final[i][0])):
    if(len(final[i][0])==1):
      final_list.append([final[i][0][j],final[i][1],final[i][2][0]])
    
    else:
      final_list.append([final[i][0][j],final[i][1],final[i][2][j]])

for i in range(len(final_list)):
  if(final_list[i][2]==0):
    final_list[i][2]=menu_list[final_list[i][0]]

json_list=[]
for i in range(len(final_list)):
  json_dict={}
  json_dict['menu_name']=final_list[i][0]
  if(final_list[i][1]==None):
    continue
  if(final_list[i][1][0]>='0' and final_list[i][1][0]<='9'):
    json_dict['option_name']=final_list[i][1]+'ml'
  else:
    json_dict['option_name']=final_list[i][1]
  
  json_dict['topping_name']=''
  json_dict['menu_count']=final_list[i][2]
  json_list.append(json_dict)
print(json_list)
print(real_list)

[['고추잡채', [0, 4]], ['양장피', [10, 13]], ['잡채밥', [19, 22]], ['중식초계냉면', [28, 34]], ['중식초계냉면', [42, 48]]]
n_word_tmp:  [['고추잡채', [0, 4]], ['양장피', [10, 13]], ['잡채밥', [19, 22]], ['중식초계냉면', [28, 34]], ['중식초계냉면', [42, 48]]]
t: 고추잡채 하나 꼬 양장피
t: 양장피 하나 꼬 잡채밥
t: 잡채밥 하나 꼬 중식초계냉면
t: 중식초계냉면 2개 주세요 중식초계냉면
t: 중식초계냉면 하나는 땅콩소스로 하나는 매운소스로 주세요 
r_real_list: ['고추잡채', '양장피', '잡채밥', '중식초계냉면', '중식초계냉면']
real_tmp_n_list: [[0, 4], [10, 13], [19, 22], [28, 34], [42, 48]]
sec: 고추잡채 하나 꼬 
sec: 양장피 하나 꼬 
sec: 잡채밥 하나 꼬 
sec: 중식초계냉면 2개 주세요 
sec: 중식초계냉면 하나는 땅콩소스로 하나는 매운소스로 주세요 
[{'menu_name': '중식초계냉면', 'option_name': '땅콩소스', 'topping_name': '', 'menu_count': 1}, {'menu_name': '중식초계냉면', 'option_name': '매운소스', 'topping_name': '', 'menu_count': 1}]
[['고추잡채'], ['양장피'], ['잡채밥'], ['중식초계냉면'], ['중식초계냉면']]


# 토핑 메뉴 추가

In [ ]:
import json
whole_dict={}
whole_tmp=[]
final_tp_dict={}


word_min=9999
final=[]
l_n_word_tmp=list(n_word_tmp)
for i in range(len(real_tmp_n_list)):
  word_min=9999
  if(i+1<len(real_tmp_n_list)):
    section=mylist[0][real_tmp_n_list[i][0]:real_tmp_n_list[i+1][0]]
  else:
    section=mylist[0][real_tmp_n_list[i][0]:]
  if(r_real_list[i] not in all_menu_dict):
    continue
  ij=all_menu_dict[r_real_list[i]]
  if(tp_menu[ij]==None):
    continue
  for j in range(len(tp_menu[ij])):
    u=section.find(tp_menu[ij][j])
    if(u!=-1):
      for k in range(len(real_tmp_n_list)):
        tt=(real_tmp_n_list[i][0]+u)-real_tmp_n_list[k][1]

        if(word_min>abs(tt)):
          word_min=abs(tt)
          if(tt>0):
            word_a=real_tmp_n_list[k][1]
            word_b=real_tmp_n_list[i][0]+u
          else:
            word_a=real_tmp_n_list[i][0]+u
            word_b=real_tmp_n_list[k][1]

          tmp_word=r_real_list[k]
        n_flag=0
        n_min=9999
        nn=0
        for k in range(len(num_list)):
          for w in range(len(num_list[k])):
            tt1=(mylist[0][word_a:word_b]+' ').find(num_list[k][w])

            if(tt1!=-1):
              if(n_min>word_b-tt1):
                n_min=word_b-tt1
                nn=k+1
                n_flag=1
        final.append([tmp_word,tp_menu[ij][j],[nn]])
      
print(final)
final_count=0
nn=0
for i in range(len(final)):
  final_count=0
  nn=0
  t1=0
  for j in range(len(l_menu_list)):
    final_count+=final[i].count(l_menu_list[j])
  if(final_count!=1):
    for k in range(len(num_list)):
      for w in range(len(num_list[k])):
        u1=final[i][0].find(num_list[k][w])
        if(u1!=-1):
          t1=final[i][2][0]
          nn=k+1
    final[i][2]=[nn,t1]
print(final)
for i in range(len(final)):
  t=[]
  for j in range(len(menu_list)):
    if(final[i][0].find(l_menu_list[j])!=-1):
      t.append(l_menu_list[j])
  final[i][0]=t
    
print(final) 
final_list=[]

for i in range(len(final)):
  n_min=9999
  nn=0
  for j in range(len(final[i][0])):
    if(len(final[i][0])==1):
      final_list.append([final[i][0][j],final[i][1],final[i][2][0]])
    
    else:
      final_list.append([final[i][0][j],final[i][1],final[i][2][j]])
            
          
          
print(final)
print(final_list)
for i in range(len(final_list)):
  if(final_list[i][2]==0):
    final_list[i][2]=menu_list[final_list[i][0]]
print(final_list)
json_tp_list=[]
for i in range(len(final_list)):
  json_tp_dict={}
  json_tp_dict['menu_name']=final_list[i][0]
  json_tp_dict['option_name']=''
  json_tp_dict['topping_name']=final_list[i][1]
  json_tp_dict['menu_count']=final_list[i][2]
  json_tp_list.append(json_tp_dict)

tmp_j=-1
dele=[]

for j in range(len(json_tp_list)):
  json_cnt=0
  tmp_j=-1
  for i in range(len(json_list)):
    print(json_tp_list[j])
    json_cnt=0
    if(json_list[i]['menu_name']==json_tp_list[j]['menu_name']):
      
      json_cnt=json_cnt+1
    if(json_list[i]['menu_count']==json_tp_list[j]['menu_count']):
      json_cnt=json_cnt+1
    if(json_cnt==2):
      json_list[i]['topping_name']=json_tp_list[j]['topping_name']
      if j not in dele:
        dele.append(j)
    print('json_tp_list del',json_tp_list)
  
for i in range(len(dele)-1,-1,-1):
  del(json_tp_list[dele[i]])

for i in range(len(json_tp_list)):
  json_list.append(json_tp_list[i])

#옵션, 토핑이 없는 메뉴들도 추가  
cpr_dict={}
for i in range(len(json_list)):
  if json_list[i]['menu_name'] not in cpr_dict:
    cpr_dict[json_list[i]['menu_name']]=json_list[i]['menu_count']
  else:
    cpr_dict[json_list[i]['menu_name']]+=json_list[i]['menu_count']
print(cpr_dict)    
l_cpr_dict=list(cpr_dict)
cpr_num=0
for i in range(len(menu_list)):
  for j in range(len(cpr_dict)):
    if(l_menu_list[i]==l_cpr_dict[j]):
      if(menu_list[l_menu_list[i]]>cpr_dict[l_cpr_dict[j]]):
        cpr_num=menu_list[l_menu_list[i]]-cpr_dict[l_cpr_dict[j]]
        json_dict={}
        json_dict['menu_name']=l_menu_list[i]
        json_dict['option_name']=''
        json_dict['topping_name']=''
        json_dict['menu_count']=cpr_num
        json_list.append(json_dict)
for i in range(len(menu_list)):
  cnt=0
  for j in range(len(json_list)):
    if(json_list[j]['menu_name']==l_menu_list[i]):
      cnt=cnt+1
  if(cnt==0):
    json_dict={}
    json_dict['menu_name']=l_menu_list[i]
    json_dict['option_name']=''
    json_dict['topping_name']=''
    json_dict['menu_count']=menu_list[l_menu_list[i]]
    json_list.append(json_dict)
print(json.dumps(json_list,ensure_ascii=False,indent='\t'))

[]
[]
[]
[]
[]
[]
{'중식초계냉면': 2}
[
	{
		"menu_name": "중식초계냉면",
		"option_name": "땅콩소스",
		"topping_name": "",
		"menu_count": 1
	},
	{
		"menu_name": "중식초계냉면",
		"option_name": "매운소스",
		"topping_name": "",
		"menu_count": 1
	},
	{
		"menu_name": "고추잡채",
		"option_name": "",
		"topping_name": "",
		"menu_count": 1
	},
	{
		"menu_name": "양장피",
		"option_name": "",
		"topping_name": "",
		"menu_count": 1
	},
	{
		"menu_name": "잡채밥",
		"option_name": "",
		"topping_name": "",
		"menu_count": 1
	}
]


# 제이슨 파일 저장


In [ ]:
import json
from collections import OrderedDict
file_data=OrderedDict()
def make_json(items):
  file_data['res_id']=22
  file_data['table_id']=99
  file_data['orders']=items
  print(json.dumps(file_data,ensure_ascii=False,indent='\t'))
  with open('/content/gdrive/My Drive/Colab Notebooks/voice_order/data/menu.json','w',encoding='utf-8') as make_file:
    json.dump(file_data,make_file,ensure_ascii=False,indent='\t')
  return json.dumps(file_data,ensure_ascii=False,indent='\t')
params=make_json(json_list)


{
	"res_id": 22,
	"table_id": 99,
	"orders": [
		{
			"menu_name": "중식초계냉면",
			"option_name": "땅콩소스",
			"topping_name": "",
			"menu_count": 1
		},
		{
			"menu_name": "중식초계냉면",
			"option_name": "매운소스",
			"topping_name": "",
			"menu_count": 1
		},
		{
			"menu_name": "고추잡채",
			"option_name": "",
			"topping_name": "",
			"menu_count": 1
		},
		{
			"menu_name": "양장피",
			"option_name": "",
			"topping_name": "",
			"menu_count": 1
		},
		{
			"menu_name": "잡채밥",
			"option_name": "",
			"topping_name": "",
			"menu_count": 1
		}
	]
}


In [ ]:
import requests
with open('/content/gdrive/My Drive/Colab Notebooks/voice_order/data/menu.json','r',encoding='utf-8') as make_file:
    uu=json.load(make_file)
print(uu)

headers={ 
        "Content-Type":"application/json"
        
}



res = requests.post(url="",
                    data=params.encode('utf-8'),
                    headers=headers)
print(res.text)


{'res_id': 22, 'table_id': 99, 'orders': [{'menu_name': '중식초계냉면', 'option_name': '땅콩소스', 'topping_name': '', 'menu_count': 1}, {'menu_name': '중식초계냉면', 'option_name': '매운소스', 'topping_name': '', 'menu_count': 1}, {'menu_name': '고추잡채', 'option_name': '', 'topping_name': '', 'menu_count': 1}, {'menu_name': '양장피', 'option_name': '', 'topping_name': '', 'menu_count': 1}, {'menu_name': '잡채밥', 'option_name': '', 'topping_name': '', 'menu_count': 1}]}
{"result":false}
